In [7]:
import pandas as pd
import json

In [11]:
# goals: 
# 1. evaluate centrality
# 2. cluster narrative function
# 3. 

In [ ]:
parsed_narr = json.load(open('../source_summaries/v2_narr_parsed/v2_train_set_narr.json'))
narr_df = pd.DataFrame(parsed_narr)
source_df = narr_df.explode('sources').dropna().reset_index(drop=True)
source_df = pd.concat([
    source_df[['url']],
    source_df['sources'].pipe(lambda s: pd.DataFrame(s.tolist()))
], axis=1)

In [50]:
source_df['Centrality'].value_counts()#['Perspective'].value_counts()

Centrality
Low               115003
Medium            101954
High               58060
N/A                   38
None                  31
Not applicable         1
Name: count, dtype: int64

In [51]:
source_df['Perspective'].value_counts()

Perspective
Informative                 103715
Authoritative                68894
Neutral                      43741
Supportive                   38837
Against                      10446
Skeptical                     8459
Defensive                      474
Analytical                     114
Critical                        70
N/A                             64
None                            41
Personal                        38
Reflective                      28
Humorous                        24
Unknown                         17
Promotional                     16
Inquisitive                     15
Sensational                     13
Varied                          11
Speculative                     10
Mixed                            7
Victim                           6
For                              4
Not applicable                   3
Grateful                         3
Uncooperative                    2
Reassuring                       2
Contrite                         2
Optimist